In [34]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ModuleNotFoundError: No module named 'sklearn'

In [31]:
glogs_final = pd.read_csv('https://raw.githubusercontent.com/tmarchok1/DS440_project/refs/heads/main/glogs_final.csv')
glogs_final.head()

# Binary encode day of week
glogs_final['day_of_week'] = glogs_final['day_of_week'].map({'Mon':0, 'Tue':0, 'Wed':0, 'Thu':0, 'Fri':1, 'Sat':1, 'Sun':1}).astype('int')
glogs_final

# One-hot encode week of season
# df_encoded = pd.get_dummies(glogs_final, columns=['v_name'])
# df_encoded

# Apply OneHotEncoder to the 'category' column
encoder = ColumnTransformer(
    transformers=[('category', OneHotEncoder(), ['category'])],
    remainder='passthrough'  # Keep other columns if there are any
)

df_encoded = encoder.fit_transform(glogs_final)

# Convert the result back to a DataFrame (optional)
df_encoded = pd.DataFrame(df_encoded, columns=encoder.get_feature_names_out())
print(df_encoded)

NameError: name 'ColumnTransformer' is not defined